<a href="https://colab.research.google.com/github/Tarun2733/game-recommendation-system/blob/main/gameModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Pandas and Numpy to preprocess data


In [24]:
import pandas as pd
import numpy as np

In [25]:
games = pd.read_csv('/content/steam.csv')
tags_df = pd.read_csv('/content/steamspy_tag_data.csv')
desc_df = pd.read_csv('/content/steam_description_data.csv')
media_df = pd.read_csv('/content/steam_media_data.csv')



In [26]:
games.head(1)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19


In [27]:
tags_df.head(1)

,appid,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,144,564,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,550


In [28]:
#create a subset of game and merge it with tags_df
subset = games[['appid','name']]
tagged = pd.merge(subset, tags_df, on='appid')


In [29]:
print(tags_df.columns.tolist())

['appid', '1980s', '1990s', '2.5d', '2d', '2d_fighter', '360_video', '3d', '3d_platformer', '3d_vision', '4_player_local', '4x', '6dof', 'atv', 'abstract', 'action', 'action_rpg', 'action_adventure', 'addictive', 'adventure', 'agriculture', 'aliens', 'alternate_history', 'america', 'animation_&_modeling', 'anime', 'arcade', 'arena_shooter', 'artificial_intelligence', 'assassin', 'asynchronous_multiplayer', 'atmospheric', 'audio_production', 'bmx', 'base_building', 'baseball', 'based_on_a_novel', 'basketball', 'batman', 'battle_royale', 'beat_em_up', 'beautiful', 'benchmark', 'bikes', 'blood', 'board_game', 'bowling', 'building', 'bullet_hell', 'bullet_time', 'crpg', 'capitalism', 'card_game', 'cartoon', 'cartoony', 'casual', 'cats', 'character_action_game', 'character_customization', 'chess', 'choices_matter', 'choose_your_own_adventure', 'cinematic', 'city_builder', 'class_based', 'classic', 'clicker', 'co_op', 'co_op_campaign', 'cold_war', 'colorful', 'comedy', 'comic_book', 'competi

In [30]:
desc_df.head(1)

,steam_appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...


In [31]:
desc_df = desc_df.rename(columns={'steam_appid': 'appid'})

In [32]:
desc_df.head(1)

,appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...


In [33]:
media_df.head(1)

,steam_appid,header_image,screenshots,background,movies
0,10,https://steamcdn-a.akamaihd.net/steam/apps/10/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN


In [34]:
games = games.merge(desc_df,on = 'appid')

In [35]:
games.shape

(27075, 21)

In [36]:
games.head(1)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,...,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,detailed_description,about_the_game,short_description
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,0,124534,3339,17612,317,10000000-20000000,7.19,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...


Select the columns that will be used:
appid,
name,
developer,
steamspy_tags



In [37]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   appid                 27075 non-null  int64  
 1   name                  27075 non-null  object 
 2   release_date          27075 non-null  object 
 3   english               27075 non-null  int64  
 4   developer             27074 non-null  object 
 5   publisher             27061 non-null  object 
 6   platforms             27075 non-null  object 
 7   required_age          27075 non-null  int64  
 8   categories            27075 non-null  object 
 9   genres                27075 non-null  object 
 10  steamspy_tags         27075 non-null  object 
 11  achievements          27075 non-null  int64  
 12  positive_ratings      27075 non-null  int64  
 13  negative_ratings      27075 non-null  int64  
 14  average_playtime      27075 non-null  int64  
 15  median_playtime    

In [38]:
print(games.columns.tolist())


['appid', 'name', 'release_date', 'english', 'developer', 'publisher', 'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags', 'achievements', 'positive_ratings', 'negative_ratings', 'average_playtime', 'median_playtime', 'owners', 'price', 'detailed_description', 'about_the_game', 'short_description']


In [39]:
games = games[['appid', 'name','developer','steamspy_tags']]

In [40]:
games.head()

,appid,name,developer,steamspy_tags
0,10,Counter-Strike,Valve,Action;FPS;Multiplayer
1,20,Team Fortress Classic,Valve,Action;FPS;Multiplayer
2,30,Day of Defeat,Valve,FPS;World War II;Multiplayer
3,40,Deathmatch Classic,Valve,Action;FPS;Multiplayer
4,50,Half-Life: Opposing Force,Gearbox Software,FPS;Action;Sci-fi


In [41]:
games.isnull().sum()

,0
appid,0
name,0
developer,1
steamspy_tags,0


In [42]:
games.dropna(inplace=True)

In [43]:
games.duplicated().sum()

np.int64(0)

In [44]:
# games['short_description'][0]

In [45]:
games['steamspy_tags'][0]

'Action;FPS;Multiplayer'

In [46]:
# convert string to list
def string_to_list(st):
  if pd.isnull(st):
    return []
  return st.lower().replace(';',' ').split()


In [47]:
# games['short_description'] = games['short_description'].apply(string_to_list)

In [48]:
games.head(1)

,appid,name,developer,steamspy_tags
0,10,Counter-Strike,Valve,Action;FPS;Multiplayer


In [49]:
#converting each column to list form
games['developer'] = games['developer'].apply(string_to_list)
games['steamspy_tags'] = games['steamspy_tags'].apply(string_to_list)
games['new_name'] = games['name'].apply(string_to_list)


In [50]:
games.head(1)

,appid,name,developer,steamspy_tags,new_name
0,10,Counter-Strike,[valve],"[action, fps, multiplayer]",[counter-strike]


In [51]:
games['tags'] = games['new_name']+games['developer']+games['steamspy_tags']

In [52]:
games.head(1)

,appid,name,developer,steamspy_tags,new_name,tags
0,10,Counter-Strike,[valve],"[action, fps, multiplayer]",[counter-strike],"[counter-strike, valve, action, fps, multiplayer]"


**Recommendation based on a single tags string**


In [53]:
# df = games[['appid','name','tags']]

In [54]:
# df.head()

In [55]:
# import nltk
# from nltk.stem.porter import PorterStemmer
# import re

# ps = PorterStemmer()
# #stemming and cleaning
# def stem(t):
#   t = re.sub('[^a-zA-Z]', ' ', t)
#   t = t.lower()
#   t = t.split()
#   lst = []
#   for word in t:
#     lst.append(ps.stem(word))

#   return " ".join(lst)

In [56]:
# ps.stem('loved')

In [57]:
# df['tags'][0]

In [58]:
# df['tags'][1]

In [59]:
# # convert list to string
# df['tags'] = df['tags'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)

In [60]:
# df.head()

In [61]:
# df['tags'] = df['tags'].apply(stem)

In [62]:
# df.head()

In [63]:
# # stemmed tags
# df['tags'][0]

**Converting each steam genre into its own column by multi label binarizer**

In [64]:
# new_df = games

In [65]:
# new_df

In [66]:
# #convert list to string
# new_df["developer"] = new_df["developer"].apply(lambda x: " ".join(x))



In [67]:
# new_df.head(1)

In [68]:
# from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer

# label = LabelEncoder()
# mlb = MultiLabelBinarizer()


In [69]:
# #label encode the developer field
# new_df["developer"] = label.fit_transform(new_df["developer"])

In [70]:
# new_df.head(1)

In [71]:
# #Splitting steamspy_tags into multiple columns using MLB
# new_df = new_df.join(pd.DataFrame(mlb.fit_transform(new_df['steamspy_tags']),
#                           columns=mlb.classes_,
#                           index=new_df.index))

In [72]:
# new_df.head(1)

In [73]:
# print(new_df.columns.tolist())

In [74]:
# #dropping columns that are of no use
# cols_to_drop = ['steamspy_tags', 'new_name', 'tags','&',"'em"]
# new_df.drop(columns=[i for i in cols_to_drop if i in new_df.columns], inplace=True)


In [75]:
# new_df.head(1)

In [76]:
# #since mlb.classes_ contain all the columns generated by mlb. loop through it to store only the remaining columns for feature extraction
# remaining = [i for i in mlb.classes_ if i in new_df.columns]
# features = ['developer']+ remaining
# #combining developer tag for creating the matrix
# v = new_df[features].values
# #reducing the weight of developer so that there is diversity in the results
# scale_v = v.copy().astype(float)
# scale_v[:,0] *= 0.5


In [77]:
# new_df.shape

In [78]:
# pd.DataFrame(v).duplicated().sum()

In [79]:
# #use nearest neighbors for picking similar data points
# from sklearn.neighbors import NearestNeighbors
# model = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='cosine')
# model.fit(scale_v)

In [80]:
# # Let's test on the first game in your dataset
# test_index = 0
# distances, indices = model.kneighbors([v[test_index]], n_neighbors=6)  # including itself

# # Print results
# print("Target Game:", new_df.iloc[test_index]['name'])
# print("\nRecommended Games:")
# for i in range(1, 6):  # skip index 0 (it will be the same game)
#     print(f"{new_df.iloc[indices[0][i]]['name']} (distance: {distances[0][i]:.4f})")


**Using Pretagged dataset to generate optimal results**

In [81]:
tagged.head(1)

,appid,name,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,Counter-Strike,144,564,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,550


**Vectorization using TF-IDF**



In [82]:
#import necessary libraries for vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [83]:
#select only the necessary columns
feat = tagged.drop(columns=['appid','name'],axis = 1)

In [84]:
#apply tf-idf
tfidf = TfidfVectorizer()
matrix = tfidf.fit_transform(feat)

In [85]:
matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 375 stored elements and shape (371, 375)>

In [86]:
#matrix computation
similarity_matrix = cosine_similarity(matrix)

In [87]:
similarity_matrix

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [88]:
#recommend function
def recommend(g):
  g = g.lower()
  idx = tagged[tagged['name'].str.lower()==g].index[0]

  #calculate distance
  d = similarity_matrix[idx]
  #select the last 6 recommendations and skip the last one
  lst = sorted(list(enumerate(d)),reverse=True, key=lambda x:x[1])[1:6]

  recoms = [tagged.iloc[i[0]]['name']for i in lst]
  return recoms

In [89]:
#test recommendation function
recommend("Counter-Strike")

['Team Fortress Classic',
 'Day of Defeat',
 'Deathmatch Classic',
 'Half-Life: Opposing Force',
 'Ricochet']